# TheGraph data access

quick explore of the RAI SAFE data

In [14]:
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'


# query_header = '''
# query {{
#     safes(first: 1000, skip:{}) {{'''

# query_tail = '''    
# }'''

# query_body = '''
#         id
#         collateral
#         debt
#         owner {
#             id
#             safes{
#                 id
#             }
#         }
#         }
# '''


query_header = '''
query {{
    redemptionPrices(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
    block # block number
    timestamp # Timestamp in seconds
    redemptionRate # The rate of change of the redemption price
    value # redemption price value
        
'''


n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    print(query)
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['redemptionPrices']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
redemptionPrices = pd.DataFrame(safes)

request 1

query {
    redemptionPrices(first: 1000, skip:0) {
    block # block number
    timestamp # Timestamp in seconds
    redemptionRate # The rate of change of the redemption price
    value # redemption price value
        
    
}
}
results 504
total 504


In [15]:
redemptionPrices

,block,redemptionRate,timestamp,value
0,11922844,0.999999980914941203165341994,1614209981,3.110957248959307245967440274
1,11884264,0.999999982822810043588985278,1613696990,3.137659234552876447657180157
2,11874001,0.999999999995517810637933345,1613560127,3.139997800189848766216163966
3,11891875,0.999999985017029119334838116,1613797781,3.133748724063381772944592024
4,11932767,1.000000009905739586119065575,1614341492,3.107189229253348771237715176
...,...,...,...,...
499,11897405,0.999999986500958129004170904,1613871323,3.130188112777539463969630196
500,11925102,0.999999986266967237489218032,1614239974,3.109613262378041784158048515
501,11937659,0.999999978657028228361932302,1614406185,3.106654270777445559270750449
502,11947334,0.999999979475285317280166361,1614535779,3.092679829664850457072193725
